# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 讀取資料

In [2]:
train_X, train_y = SVHN2.traindata(Float32, 1:20000);
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000);

In [3]:
train_y = onehotbatch(train_y, 1:10);
test_y = onehotbatch(test_y, 1:10);

In [4]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true);
test = DataLoader(test_X, test_y, batchsize=batchsize);

## CNN 模型

In [5]:
# write your model here
model = Chain(
    Conv((3, 3), 3=>32, relu),
    MaxPool((2,2), pad=(0,0), stride=(2,2)),
    Conv((3, 3), 32=>32, relu),
    MaxPool((2,2), pad=(0,0), stride=(2,2)),
    Conv((3, 3), 32=>64, relu),
    MaxPool((2,2), pad=(0,0), stride=(2,2)),
    flatten,
    Dense(256, 10),
    softmax)

Chain(Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0), stride = (2, 2)), flatten, Dense(256, 10), softmax)

In [ ]:
using CuArrays
model = model |> gpu
train_X = train_X |> gpu
train_y = train_y |> gpu
test_X = test_X |> gpu
test_y = test_y |> gpu

### Loss function

In [6]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

In [7]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

### Training

In [8]:
epochs = 5
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2838044f0
test_loss() = 2.2665455f0
test_loss() = 2.2556925f0
test_loss() = 2.2490048f0


┌ Info: Epoch 2
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2608497f0
test_loss() = 2.2426667f0
test_loss() = 2.2278023f0
test_loss() = 2.199329f0


┌ Info: Epoch 3
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2382977f0
test_loss() = 2.1749516f0
test_loss() = 2.1592903f0
test_loss() = 2.128281f0


┌ Info: Epoch 4
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.1833518f0
test_loss() = 2.1235118f0
test_loss() = 2.1144624f0
test_loss() = 2.1056795f0


┌ Info: Epoch 5
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.1747859f0
test_loss() = 2.100937f0
test_loss() = 2.09686f0
test_loss() = 2.0811417f0


### Evaluation

In [9]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [10]:
accuracy(test_X, test_y)

0.383